In [1]:
import xml, os, pandas as pd, glob
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import fromstring, ElementTree
import requests, re
import time
import requests
import string
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

In [2]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
npp_ts = pd.read_csv('npp_timespans.csv', sep = ';')

In [3]:
os.chdir("D://Scriptie//Data//csv-metadata")
list_csv = glob.glob('*.csv')

In [4]:
def GetTS(newspaper_title):
    sy = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'sy'].iloc[0]
    ey = npp_ts.loc[npp_ts['npp'] == newspaper_title, 'ey'].iloc[0]
    t = ey - sy
    return t

In [5]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [6]:
url1= "http://services.kb.nl/mdo/oai?verb=GetRecord&identifier=DDD:"
url3 = "&metadataPrefix=didl"

In [7]:
def DateXML(soup_element):
    t = str(soup_element.find_all("date")[0])[6:10]
    return t

In [8]:
def OCRXML(soup_element):
    t = soup_element.find_all("OCRConfidencelevel")
    t = [str(i) for i in t]
    t = [i.replace('<OCRConfidencelevel>', '') for i in t]
    t = [i.replace('<OCRConfidencelevel/>', '') for i in t]
    t = [i.replace('</OCRConfidencelevel>', '') for i in t]
    t = [i for i in t if i]
    t = [int(i.replace('0.', '')) for i in t]
    if len(t) > 0:
        t = (sum(t) / len(t)) / 1000
    return t

In [9]:
def GetTitleNppYear(newspaper_title, year):
    fn = [fn for fn in list_csv if fn[0:4] == newspaper_title][0]
    
    df = pd.read_csv(fn, sep = '\t')
    df['id'] = df.id.str[:-6]
    df['date'] = df.date.str.slice(0,4)
    df= df[df['id'].str[0:3] == 'ddd']
    
    df = df.drop_duplicates(subset='id', keep="first")
    df = df[df['date'] == str(year)]
    if len(df) > 0:    

        df = df.sample(n=30)
        df = df.reset_index(drop=True)

        titles_list = list()

        for issue in list(df.id):
            url = url1 + issue + url3
            result = requests.get(url)
            content = result.content
            soup = BeautifulSoup(content, "xml")
            list_titles = soup.find_all("title")
            list_titles = [str(i) for i in list_titles]
            list_titles = [i.replace('<title>', '') for i in list_titles]
            list_titles = [i.replace('</title>', '') for i in list_titles]
            list_titles = [clean_and_split_str(i) for i in list_titles]

            for title in list_titles:
                t = [title, OCRXML(soup), issue]
                titles_list.append(t)
    if len(df) == 0:
        titles_list = [[year, 'no titles', 'in issue']]
    
    return titles_list
        
    

In [33]:
npp_df = pd.DataFrame()

for year in list(range(1823,1869)):
    dl = GetTitleNppYear('prda', year)
    dl = pd.DataFrame(dl, columns = ['title', 'ocr', 'issue'])
    dl['year'] = year
    npp_df = npp_df.append(dl)

In [35]:
npp_df.to_csv("prda_titles.csv")

In [34]:
npp_df

,title,ocr,issue,year
0,provinciale drentsche en asser courant,0.73975,ddd:010783923:mpeg21,1823
1,wet van den 21sten augustus 1822 houdende bela...,0.73975,ddd:010783923:mpeg21,1823
2,wet van den 21sten augustus 1822 houdende bela...,0.73975,ddd:010783923:mpeg21,1823
3,notificatie,0.73975,ddd:010783923:mpeg21,1823
4,de ontvanger van het pensioenfonds voor de amb,0.73975,ddd:010783923:mpeg21,1823
5,s gravenhage den 25 juni,0.73975,ddd:010783923:mpeg21,1823
6,brussel den 26 juni,0.73975,ddd:010783923:mpeg21,1823
7,rotterdam den 27 juni,0.73975,ddd:010783923:mpeg21,1823
8,frankfort den 19 juni,0.73975,ddd:010783923:mpeg21,1823
9,londen den 24 juni,0.73975,ddd:010783923:mpeg21,1823


In [28]:
for npp in tqdm(list(npp_ts.npp)):
    npp_df = pd.DataFrame()

    for year in list(range(int(npp_ts[npp_ts.npp == npp].sy), int(npp_ts[npp_ts.npp == npp].ey))):
        dl = GetTitleNppYear('nesc', year)
        dl = pd.DataFrame(dl, columns = ['title', 'ocr', 'issue'])
        dl['year'] = year
        npp_df = npp_df.append(dl)
    
    fn = npp + "_titles.csv"
    npp_df.to_csv(fn)


 52%|████████████████████████████████████████████████████████████████████████████████████████                                                                                | 11/21 [1:20:15<1:27:39, 525.97s/it]

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [11]:
npp_df = pd.DataFrame()

for year in list(range(1815,1915)):
    dl = GetTitleNppYear('leco', year)
    dl = pd.DataFrame(dl, columns = ['title', 'ocr', 'issue'])
    dl['year'] = year
    npp_df = npp_df.append(dl)

In [14]:
npp_df.to_csv('leco-titles.csv', index=False)

In [16]:
for c,i in list(npp_df.title):
    if "buitenland" in i:
        print(i)

buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
buitenlandsche scheepvaart
b

buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsch scheepvaart te harlingen
buitenlandsch scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
buitenlandsche scheepvaart te harlingen
bu

buitenland algemeen overzigt 1 maart
buitenland algemeen overzigt 1 maart
buitenland algemeen overzigt 7 april
buitenland algemeen overzigt 7 april
buitenland algemeen overzigt 25 september
buitenland algemeen overzigt 25 september
buitenland algemeen overzigt 16 april
buitenland algemeen overzigt 16 april
buitenland algemeen overzigt 9 october
buitenland algemeen overzigt 9 october
buitenland algemeen overzigt 27 maart
buitenland algemeen overzigt 27 maart
buitenland algemeen overzigt 26 september
buitenland algemeen overzigt 26 september
buitenland algemeen overzigt 17 junij
buitenland algemeen overzigt 17 junij
buitenland algemeen overzigt 15 maart
buitenland algemeen overzigt 15 maart
buitenland algemeen overzigt 19 februarij
buitenland algemeen overzigt 19 februarij
buitenland algemeen overzigt 20 junij
buitenland algemeen overzigt 20 junij
buitenland algemeen overzigt 9 december
buitenland algemeen overzigt 9 december
buitenland algemeen overzigt 13 october
buitenland algemeen ov

buitenland algemeen overzicht 18 augustus
buitenland algemeen overzicht 18 augustus
buitenland algemeen overzicht 4 augustus
buitenland algemeen overzicht 4 augustus
buitenland algemeen overzicht 1 october
buitenland algemeen overzicht 1 october
buitenland algemeen overzicht 5 mei
buitenland algemeen overzicht 5 mei
buitenland algemeen overzicht 12 juni
buitenland algemeen overzicht 12 juni
buitenland algemeen overzicht 4 juni
buitenland algemeen overzicht 4 juni
buitenland algemeen overzicht 13 juni
buitenland algemeen overzicht 13 juni
buitenland algemeen overzicht 13 october
buitenland algemeen overzicht 13 october
buitenland algemeen overzicht 1 mei
buitenland algemeen overzicht 1 mei
buitenland algemeen overzicht 17 september
buitenland algemeen overzicht 17 september
buitenland algemeen overzicht 11 augustus
buitenland algemeen overzicht 11 augustus
buitenland algemeen overzicht 3 augustus
buitenland algemeen overzicht 3 augustus
buitenland algemeen overzicht 23 december
buitenla

buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 19 januari
buitenlandsche berichten
buitenlandsch overzicht 17 april
buitenlandsche berichten
buitenlandsch overzicht 17 april
buitenlandsche berichten
buitenlandsch overzicht 16 september
buitenlandsche berichten
buitenlandsch overzicht 16 september
buitenlandsche berichten
buitenlandsch overzicht 17 juli
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 17 juli
buitenlandsche berichten
buitenlandsch overzicht 20 december
buitenlandsche berichten
buitenlandsch overzicht 20 december
buitenlandsche berichten
buitenlandsch overzicht 9 october
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 9 october
buitenlandsche berichten
buitenlandsch overzicht 8 september
buitenlandsche berichten
buitenlandsch overzicht 8 september
buitenlandsche berichten
buitenlandsch overzicht 3 januari
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 3 januari
buiten

buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 23 october
buitenlandsche berichten
buitenlandsch overzicht 5 december
buitenlandsche berichten
buitenlandsche berichten
buitenlandsche zaken
buitenlandsch overzicht 5 december
buitenlandsche berichten
buitenlandsche zaken
buitenlandsch overzicht 12 april
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 12 april
buitenlandsche berichten
buitenlandsch overzicht 11 november
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 11 november
buitenlandsche berichten
buitenlandsch overzicht 4 januari
de buitenlandsche vijand
buitenlandsche berichten
buitenlandsch overzicht 4 januari
de buitenlandsche vijand
buitenlandsche berichten
buitenlandsch overzicht 23 april
buitenlandsche berichten
buitenlandsch overzicht 23 april
buitenlandsche berichten
buitenlandsch overzicht 29 mei
buitenlandsche berichten
buitenlandsch overzicht 29 mei
buitenlandsche berichten
buitenlandsch overz

buitenlandsch overzicht 6 december
buitenlandsche berichten
buitenlandsch overzicht 6 december
buitenlandsche berichten
buitenlandsch overzicht 4 maart
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 4 maart
buitenlandsche berichten
buitenlandsch overzich 4 september
buitenlandsche berichten
buitenlandsch overzich 4 september
buitenlandsche berichten
buitenlandsch overzicht 28 mei
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 28 mei
buitenlandsche berichten
buitenlandsch overzicht 12 maart
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 12 maart
buitenlandsche berichten
buitenlandsch overzicht 15 februari
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 15 februari
buitenlandsche berichten
buitenlandsch overzicht 27 november
buitenlandsche berichten
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 27 november
buitenlandsche berichten
buitenlandsche berichten

buitenlandsch overzicht 10 maart
buitenlandsche berichten
buitenlandsch overzicht 3 october de oorlog
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 3 october de oorlog
buitenlandsche berichten
buitenlandsch overzicht 22 februari
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 22 februari
buitenlandsche berichten
buitenlandsch overzicht 5 juli
buitenlandsche berichten
buitenlandsch overzicht 5 juli
buitenlandsche berichten
buitenlandsch overzicht 6 februari
buitenlandsche berichten
buitenlandsche berichten
buitenlandsch overzicht 6 februari
buitenlandsche berichten
buitenlandsch overzicht 20 september
buitenlandsche berichten
buitenlandsch overzicht 20 september
buitenlandsche berichten
buitenlandsch overzicht 1 september
buitenlandsche berichten
buitenlandsch overzicht 1 september
buitenlandsche berichten
buitenlandsche berichten
buitenlandsche berichten
buitenlandsche berichten
buitenlandsche berichten
buitenlandsche berichten
buit

In [17]:
npp_df

,title,ocr,issue,year
0,leeuwarder courant,0.748750,ddd:010580065:mpeg21,1815
1,publicatie,0.748750,ddd:010580065:mpeg21,1815
2,napels den 28 januarij,0.748750,ddd:010580065:mpeg21,1815
3,napels den 28 januarij,0.748750,ddd:010580065:mpeg21,1815
4,napels den 28 januarij,0.748750,ddd:010580065:mpeg21,1815
5,advertentie,0.748750,ddd:010580065:mpeg21,1815
6,advertentie,0.748750,ddd:010580065:mpeg21,1815
7,advertentie,0.748750,ddd:010580065:mpeg21,1815
8,beursprijs der effecten amsterdam den 18 febru...,0.748750,ddd:010580065:mpeg21,1815
9,advertentie,0.748750,ddd:010580065:mpeg21,1815
